In [1]:
import sagemaker
import boto3
import os
import wandb
from sagemaker.pytorch import PyTorch

In [2]:
try:
    sagemaker_session = sagemaker.Session()
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_permissions')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name Shivam to get Role path.


In [1]:
from datetime import datetime

bucket = "ai-sagemaker-weights"
base_job_name = "viton-hd"
checkpoint_suffix = "resume-weights" # datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
checkpoint_s3_path = f"s3://{bucket}/{base_job_name}-{checkpoint_suffix}"
print(checkpoint_s3_path)

In [4]:
source_dir = os.getcwd()
wandb.sagemaker_auth(path=source_dir)

estimator = PyTorch(entry_point='sage_train.sh',
                    source_dir=source_dir,
                    role=role,
                    py_version='py38',
                    framework_version='1.11.0',
                    base_job_name=base_job_name,
                    checkpoint_s3_uri=checkpoint_s3_path,
                    # input_mode='FastFile',
                    instance_count=1,
                    instance_type='ml.g5.xlarge',
                    use_spot_instances=True,  # Use a spot instance
                    max_run=3*24*60*60,  # Max training time
                    max_wait=3*24*60*60,  # Max training time + spot waiting time seconds
                    hyperparameters={
                        "epochs": 100,
                        "batch_size": 2,
                        "project": "VITON-HD",
                        "load_height": 1024,
                        "load_width": 768,
                        "use_wandb": True,
                        "workers": 8,
                        "dataset_dir": "/home/ubuntu/data/datadrive/docker_data/",
                        "checkpoint_dir": "checkpoints/",
                    })

In [ ]:
estimator.fit({'train': 's3://ai-sagemaker-datasets/viton_tops/'}, wait=True)